## NYC Taxi trips, with [Datashader](https://github.com/bokeh/datashader),  [HoloViews](http://holoviews.org), [GeoViews](https://github.com/ioam/geoviews/blob/master/README.md#installation) and [ParamNB](https://anaconda.org/jbednar/paramnb)

The InteractiveImage command provided with [datashader](https://github.com/bokeh/datashader) makes it simple to make an interactive plot of a very large dataset, but very often one will want to add additional interactive controls to filter your data, select columns for plotting, etc., which is not supported by InteractiveImage.  One way to do that is to use [ParamNB](https://anaconda.org/jbednar/paramnb) to instantiate some parameters and then have it run the subsequent cell whenever one of those parameters changes (via ``paramnb.Widgets(...,next_n=1)``).

This notebook illustrates a cleaner way using a [HoloViews](http://holoviews.org) stream to connect the widgets and the plot. Requires ``conda install -c ioam/label/dev holoviews paramnb`` and installing [GeoViews](https://github.com/ioam/geoviews/blob/master/README.md#installation) (which is only important for the map tile support).

## Setup

In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb
import pandas as pd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY

hv.notebook_extension('bokeh')

%time df = pd.read_csv('data/nyc_taxi.csv', usecols = ['passenger_count', \
                       'pickup_x', 'pickup_y', 'dropoff_x', 'dropoff_y'])
df.tail()

# Interactive plot

In [ ]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/'
                                   'World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))
tile_options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)  

passenger_counts = sorted(df.passenger_count.unique().tolist())

class Options(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.ObjectSelector(default=1,          objects=passenger_counts)
    
    def make_plot(self, x_range=None, y_range=None, **kwargs):
        map_tiles = tiles(style=dict(alpha=self.alpha), plot=tile_options) 

        df_filt = df[df.passenger_count==self.passengers]
        points = hv.Points(gv.Dataset(df_filt, kdims=[self.plot+'_x', self.plot+'_y'], vdims=[]))
        taxi_trips = datashade(points, width=800, height=475, x_sampling=1, y_sampling=1, 
                               cmap=self.colormap, element_type=gv.Image,
                               dynamic=False, x_range=x_range, y_range=y_range)
                
        return map_tiles * taxi_trips

selector = Options(name="")
paramnb.Widgets(selector, callback=selector.update)
hv.DynamicMap(selector.make_plot, kdims=[], streams=[selector, RangeXY()])

If you are viewing a static copy of this page through Anaconda Cloud, the interactive controls will not be visible, but you can download a copy of this notebook and run it through Jupyter notebook for the interactive version.  You can also view the controls and the image as a deployable dashboard using [Jupyter Dashboards](https://github.com/jupyter/dashboards), which can be installed separately using ``conda install -c conda-forge jupyter_dashboards``.  Jupyter Dashboards is a Jupyter extension that lets you choose which cells to publish to a dashboard layout, with the result like this [screenshot](https://raw.githubusercontent.com/bokeh/datashader/master/docs/images/nyc_taxi-paramnb.png) that can be deployed as a standalone server.